In [1]:
colab=False #True: if in colab, Flase: if in local (pls change accordinngly!)

In [2]:
if colab:
    from google.colab import drive
    drive.mount('/content/drive')

In [3]:
if colab:
    #Add a shortcut of project folder to your drive's home for this to work
    !ls '/content/drive/MyDrive/Traffic'
    root = '/content/drive/MyDrive/Traffic'
else:
    import os
    root = os.getcwd()
root

'/home/skycam/sample_projs/FRCNN/delivered/FastRCNN_Traffic'

In [4]:
#!unzip "/content/drive/MyDrive/Traffic/Traffic_FRCNN.zip" -d "/content/drive/MyDrive/Traffic/"

Uncomment above line after putting zip file inside Traffic folder. Create another notebook and put above cells in it do this unzipping

In [5]:
if colab:
    import sys
    sys.path.append(root)

In [6]:
if colab:
    !pip3 install -r '/content/drive/MyDrive/Traffic/requirements.txt'
else:
    !pip3 install -r requirements.txt

In [7]:
import os
from utilities.data_utils.Dataset import FacialDataset, get_transform
from utilities.utils import collate_fn
from utilities.train_eval.engine import train_one_epoch, evaluate, get_model_result
import glob

import nvidia_smi # for python 3, you need nvidia-ml-py3 library

import torch
torch.cuda.empty_cache()
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.rpn import AnchorGenerator, RPNHead

In [8]:
PATH = 'Weight.pth'

output_image_folder = 'output'

proj = 'sign' # 'sign','fabric' (put ur project name here)

if proj == 'sign':
    classes = ['warning', 'prohibitory', 'mandatory']
elif proj == 'fabric':
    classes = ['Hole', 'Line', 'Stain']
else:
    classes = ['bg']

num_classes = len(classes)+1  # n class + background

batch_size = 3

num_epochs = 30 

In [9]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
#handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
#info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
#print("Total memory:", info.total)
#print("Free memory:", info.free)
#print("Used memory:", info.used)

In [10]:
dataset_train = FacialDataset(root+'/data/train', get_transform(horizontal_flip=True),classes=classes)
dataset_test = FacialDataset(root+'/data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_train = torch.utils.data.DataLoader(
        dataset_train, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=0,
        collate_fn=collate_fn)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)


In [12]:
anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                        aspect_ratios=([1.0, 1.0, 1.0, 1.0], 
                                                     [0.8, 1.0, 1.0, 1.0], 
                                                     [1.0, 0.8, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0],
                                                     [1.0, 1.0, 1.0, 1.0]))

In [13]:
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


In [14]:
if os.path.isfile(PATH):
    model.load_state_dict(torch.load(PATH))

model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005,
                            momentum=0.9, weight_decay=0.0005)
# and a learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                               step_size=3,
                                               gamma=0.1)

## Training

In [15]:
for epoch in range(num_epochs):
    #res = nvidia_smi.nvmlDeviceGetUtilizationRates(handle)
    #print(f'gpu: {res.gpu}%, gpu-mem: {res.memory}%')

    train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=1)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

print("Training complete!")

Epoch: [0]  [ 0/11]  eta: 0:01:04  lr: 0.000505  loss: 0.6529 (0.6529)  loss_classifier: 0.0254 (0.0254)  loss_box_reg: 0.0108 (0.0108)  loss_objectness: 0.0854 (0.0854)  loss_rpn_box_reg: 0.5313 (0.5313)  time: 5.8782  data: 2.0336  max mem: 3747
Epoch: [0]  [ 1/11]  eta: 0:00:35  lr: 0.001004  loss: 0.4463 (0.5496)  loss_classifier: 0.0254 (0.0325)  loss_box_reg: 0.0108 (0.0198)  loss_objectness: 0.0701 (0.0778)  loss_rpn_box_reg: 0.3079 (0.4196)  time: 3.5635  data: 1.2454  max mem: 4011
Epoch: [0]  [ 2/11]  eta: 0:00:24  lr: 0.001503  loss: 0.4463 (0.5043)  loss_classifier: 0.0395 (0.0361)  loss_box_reg: 0.0289 (0.0231)  loss_objectness: 0.0784 (0.0780)  loss_rpn_box_reg: 0.3079 (0.3671)  time: 2.7759  data: 0.9492  max mem: 4011
Epoch: [0]  [ 3/11]  eta: 0:00:18  lr: 0.002003  loss: 0.4423 (0.4888)  loss_classifier: 0.0346 (0.0357)  loss_box_reg: 0.0231 (0.0231)  loss_objectness: 0.0701 (0.0735)  loss_rpn_box_reg: 0.3079 (0.3564)  time: 2.3568  data: 0.8196  max mem: 4011
Epoch: [

Epoch: [2]  [ 0/11]  eta: 0:00:10  lr: 0.005000  loss: 0.3830 (0.3830)  loss_classifier: 0.0304 (0.0304)  loss_box_reg: 0.0287 (0.0287)  loss_objectness: 0.0373 (0.0373)  loss_rpn_box_reg: 0.2866 (0.2866)  time: 0.9938  data: 0.1802  max mem: 4012
Epoch: [2]  [ 1/11]  eta: 0:00:09  lr: 0.005000  loss: 0.3830 (0.6319)  loss_classifier: 0.0304 (0.0508)  loss_box_reg: 0.0287 (0.0332)  loss_objectness: 0.0373 (0.0519)  loss_rpn_box_reg: 0.2866 (0.4960)  time: 0.9997  data: 0.1911  max mem: 4012
Epoch: [2]  [ 2/11]  eta: 0:00:09  lr: 0.005000  loss: 0.5512 (0.6050)  loss_classifier: 0.0345 (0.0454)  loss_box_reg: 0.0303 (0.0323)  loss_objectness: 0.0399 (0.0479)  loss_rpn_box_reg: 0.4464 (0.4795)  time: 1.0497  data: 0.2445  max mem: 4012
Epoch: [2]  [ 3/11]  eta: 0:00:08  lr: 0.005000  loss: 0.4955 (0.5776)  loss_classifier: 0.0304 (0.0389)  loss_box_reg: 0.0287 (0.0284)  loss_objectness: 0.0373 (0.0441)  loss_rpn_box_reg: 0.4262 (0.4662)  time: 1.0346  data: 0.2538  max mem: 4012
Epoch: [

KeyboardInterrupt: 

In [16]:
# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)

## Evaluate

In [17]:
evaluate(model, data_loader_test, device=device)

n_threads =  1
creating index...
index created!
Test:  [0/6]  eta: 0:00:00  model_time: 0.1013 (0.1013)  evaluator_time: 0.0013 (0.0013)  time: 0.1320  data: 0.0280  max mem: 4012
Test:  [5/6]  eta: 0:00:00  model_time: 0.1030 (0.1066)  evaluator_time: 0.0013 (0.0013)  time: 0.1354  data: 0.0261  max mem: 4012
Test: Total time: 0:00:00 (0.1356 s / it)
Averaged stats: model_time: 0.1030 (0.1066)  evaluator_time: 0.0013 (0.0013)
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.302
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.572
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.136
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.400
 Average Recall     

In [18]:
torch.cuda.synchronize()
# create directory for saving the model
print("Saving model...")
torch.save(model.state_dict(), PATH)
print("Model saving complete!")
nvidia_smi.nvmlShutdown()

Saving model...
Model saving complete!


## Inference 
Repeating all the process to ensure nothing is taken from training stage
Check output folder for results

In [19]:
torch.cuda.empty_cache()
nvidia_smi.nvmlInit()
handle = nvidia_smi.nvmlDeviceGetHandleByIndex(0)
info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
print("Total memory:", info.total)
print("Free memory:", info.free)
print("Used memory:", info.used)

dataset_test = FacialDataset(root+'/data/test', get_transform(horizontal_flip=False),classes=classes)

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=0,
    collate_fn=collate_fn)


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

anchor_generator = AnchorGenerator(sizes=((32,), (24, ), (24, ), (16,), (8, )),
                                   aspect_ratios=([1.0, 1.0, 1.0, 1.0],
                                                  [0.8, 1.0, 1.0, 1.0],
                                                  [1.0, 0.8, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0],
                                                  [1.0, 1.0, 1.0, 1.0]))
model.rpn.anchor_generator = anchor_generator
model.rpn.head = RPNHead(256, anchor_generator.num_anchors_per_location()[0])
# get the number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load(PATH))
model.to(device)

# create output directory
# if output directory exists, delete existing files
if not os.path.exists(output_image_folder):
    os.mkdir(output_image_folder)
else:
    files = glob.glob(output_image_folder + '/*')
    for f in files:
        os.remove(f)
# write testing result to output folder
for img_idx, batch_sampler in enumerate(data_loader_test):
    img_test = batch_sampler[0][0]
    target_test = batch_sampler[1][0]
    i = target_test["image_id"].item()
    get_model_result(img_test, model, target_test, i, device, location=output_image_folder, threshold=0.2,classes=classes)

print("Testing complete!")

torch.cuda.synchronize()
nvidia_smi.nvmlShutdown()

Total memory: 8505131008
Free memory: 6600261632
Used memory: 1904869376
initial boxes: ['warning', 'mandatory', 'mandatory', 'prohibitory', 'prohibitory', 'warning']
initial boxes: ['warning', 'mandatory', 'mandatory', 'prohibitory', 'warning', 'prohibitory']
initial boxes: ['mandatory', 'warning', 'prohibitory', 'mandatory', 'mandatory', 'prohibitory', 'warning']
initial boxes: ['mandatory', 'mandatory', 'mandatory', 'warning', 'warning', 'prohibitory', 'warning', 'prohibitory', 'mandatory', 'prohibitory', 'warning', 'mandatory', 'mandatory', 'prohibitory', 'mandatory', 'mandatory', 'warning', 'warning', 'warning']
initial boxes: ['warning', 'mandatory', 'prohibitory']
initial boxes: ['mandatory', 'warning', 'prohibitory', 'warning', 'prohibitory', 'mandatory', 'warning', 'mandatory', 'prohibitory', 'mandatory', 'warning']
Testing complete!


Check output folder to see the result images